In [ ]:
# Convert data to python arrays

import numpy as np
import os
from enum import Enum

class BoardFields(Enum):
    EMPTY = 0
    PLAYER1 = 1
    PLAYER2 = 2

NO_MOVE = -1000

boards = []
y = []

def read_line(string):
    values = string.split(",")
    moves_string = values[0]
    evaluations = [int(values[i]) for i in range(1, 8)]
    indexes = [0 for _ in range(7)]
    matrix = np.full((7, 6), BoardFields.EMPTY.value, dtype="int8")
    turn = BoardFields.PLAYER1

    for move_string in moves_string:
        move = int(move_string) - 1
        matrix[move, indexes[move]] = turn.value
        indexes[move] += 1
        turn = (
            BoardFields.PLAYER1 if turn == BoardFields.PLAYER2 else BoardFields.PLAYER2
        )

    for i in range(len(evaluations)):
        if evaluations[i] == NO_MOVE:
            continue
        child_matrix = np.copy(matrix)
        child_matrix[i, indexes[i]] = turn.value
        boards.append(child_matrix)
        y.append(evaluations[i])


for filename in os.listdir(os.getcwd() + "/data"):
    path = os.getcwd() + "/data/" + filename
    try:
        with open(path, "r") as file:
            for line in file:
                read_line(line.strip())
    except FileNotFoundError:
        print("File not found.")
    except IOError:
        print("Error reading the file.")

np.save(os.getcwd() + "/boards.npy", boards)
np.save(os.getcwd() + "/y.npy", y)


In [ ]:
# Convert python arrays into model input

import numpy as np
import os
from enum import Enum

from convert_board import convert_board

class BoardFields(Enum):
    EMPTY = 0
    PLAYER1 = 1
    PLAYER2 = 2

boards = np.load(os.getcwd() + "/boards.npy")
positions = []
my_lines = []
enemy_lines = []

length = len(boards)
increment = 1
count = 0
percentage = 0
for board in boards:
    converted_board = convert_board(board)

    positions.append(converted_board[0])
    my_lines.append(converted_board[1])
    enemy_lines.append(converted_board[2])

    count += 1
    if count > percentage / 100 * length:
        percentage += increment
        print(percentage, "%")


np.save(os.getcwd() + "/positions.npy", positions)
np.save(os.getcwd() + "/my_lines.npy", my_lines)
np.save(os.getcwd() + "/enemy_lines.npy", enemy_lines)


In [35]:
#Create model

import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Concatenate, Input
from tensorflow.keras.models import Model
import datetime as dt


input_shape1 = (2, 7, 6)
input_shape2 = (3,)
input_shape3 = (3,)

input1 = Input(shape=input_shape1, name="regression with lines")
input2 = Input(shape=input_shape2)
input3 = Input(shape=input_shape3)

flatten1 = Flatten()(input1)

concatenated = Concatenate()([flatten1, input2, input3])

dense1 = Dense(units=12, activation='relu')(concatenated)
dense2 = Dense(units=6, activation='relu')(dense1)
dense3 = Dense(units=2, activation='relu')(dense2)
output = Dense(units=1, activation='sigmoid')(dense3)


model = Model(inputs=[input1, input2, input3], outputs=output)
model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mae"])
model.summary()

Model: "functional_47"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ regression with     │ (None, 2, 7, 6)   │          0 │ -                 │
│ lines (InputLayer)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_27          │ (None, 84)        │          0 │ regression with   │
│ (Flatten)           │                   │            │ lines[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_46      │ (None, 3)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_47      │ (None, 3)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_27      │ (None, 90)        │          0 │ flatten_27[0][0], │
│ (Concatenate)       │                   │            │ input_layer_46[0… │
│                     │                   │            │ input_layer_47[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_107 (Dense)   │ (None, 12)        │      1,092 │ concatenate_27[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_108 (Dense)   │ (None, 6)         │         78 │ dense_107[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_109 (Dense)   │ (None, 2)         │         14 │ dense_108[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_110 (Dense)   │ (None, 1)         │          3 │ dense_109[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,187 (4.64 KB)

 Trainable params: 1,187 (4.64 KB)

 Non-trainable params: 0 (0.00 B)

In [33]:
# Create model

import keras

input_shape1 = (2, 7, 6)
input_shape2 = (3,)
input_shape3 = (3,)
array_input = keras.layers.Input(shape = (2, 7, 6), name = "array_input")

input1 = keras.layers.Input(shape=input_shape1, name="regression: board and lines")
input2 = keras.layers.Input(shape=input_shape2)
input3 = keras.layers.Input(shape=input_shape3)

flatten1 = keras.layers.Flatten()(input1)

concatenated = keras.layers.Concatenate()([flatten1, input2, input3])

# Dense layers for processing concatenated inputs
dense1 = keras.layers.Dense(1024, activation = "relu")(concatenated)
dense2 = keras.layers.Dense(512, activation = "relu")(dense1)
dense3 = keras.layers.Dense(256, activation = "relu")(dense2)

# Output layer
output = keras.layers.Dense(1, activation = "linear", name = "output")(dense3)


model = keras.models.Model(inputs=[input1, input2, input3], outputs=output)
model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mae"])
model.summary()

Model: "functional_45"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ regression: board   │ (None, 2, 7, 6)   │          0 │ -                 │
│ and lines           │                   │            │                   │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_26          │ (None, 84)        │          0 │ regression: board │
│ (Flatten)           │                   │            │ and lines[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_44      │ (None, 3)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_45      │ (None, 3)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_26      │ (None, 90)        │          0 │ flatten_26[0][0], │
│ (Concatenate)       │                   │            │ input_layer_44[0… │
│                     │                   │            │ input_layer_45[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_104 (Dense)   │ (None, 1024)      │     93,184 │ concatenate_26[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_105 (Dense)   │ (None, 512)       │    524,800 │ dense_104[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_106 (Dense)   │ (None, 256)       │    131,328 │ dense_105[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 1)         │        257 │ dense_106[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 749,569 (2.86 MB)

 Trainable params: 749,569 (2.86 MB)

 Non-trainable params: 0 (0.00 B)

In [51]:
# Create model

import keras

input_shape1 = (3,)
input_shape2 = (3,)

input1 = keras.layers.Input(shape=input_shape1, name="regression: lines")
input2 = keras.layers.Input(shape=input_shape2)

concatenated = keras.layers.Concatenate()([input1, input2])

# Output layer
output = keras.layers.Dense(1, activation = "linear", name = "output")(concatenated)


model = keras.models.Model(inputs=[input1, input2], outputs=output)
model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mae"])
model.summary()

Model: "functional_51"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ regression: lines   │ (None, 3)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_49      │ (None, 3)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_29      │ (None, 6)         │          0 │ regression:       │
│ (Concatenate)       │                   │            │ lines[0][0],      │
│                     │                   │            │ input_layer_49[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 1)         │          7 │ concatenate_29[0… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 7 (28.00 B)

 Trainable params: 7 (28.00 B)

 Non-trainable params: 0 (0.00 B)

Original Weights of dense_layer:
 [[ 0.8956107 ]
 [-0.89115757]
 [-0.38173008]
 [ 0.21355939]
 [-0.49246132]
 [ 0.034989  ]]
Original Biases of dense_layer:
 [0.]


In [56]:
# Save model

import datetime as dt
import os

model_name = "lines"

if not os.path.exists(f"models/{model_name}"):
    os.makedirs(f"models/{model_name}")
model.save(f"models/{model_name}/{dt.datetime.today().strftime('%Y-%m-%d_%H-%M-%S')}.keras")

In [52]:
# Train model

import numpy as np
import tensorflow as tf
from datetime import datetime
from os import listdir
from os.path import isfile, join

ys = np.load("y.npy")
positions = np.load("positions.npy")
my_lines = np.load("my_lines.npy")
enemy_lines = np.load("enemy_lines.npy")

while True:
    model_version = sorted([f for f in listdir(f"models/{model_name}") if isfile(join(f"models/{model_name}", f))])[-1]
    model = tf.keras.models.load_model(f"models/{model_name}/{model_version}")
    model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mae"])

    # TODO: Add positions as input
    model.fit([my_lines, enemy_lines], ys, epochs = 1, batch_size = 128, validation_split = 0.2)

    model.save(f"models/{model_name}/{datetime.today().strftime('%Y-%m-%d_%H-%M-%S')}.keras")

15018/15018 ━━━━━━━━━━━━━━━━━━━━ 34s 2ms/step - loss: 34.7283 - mae: 4.6791 - val_loss: 74.7495 - val_mae: 7.2002
Original Weights of dense_layer:
 [[-0.07561751]
 [-0.10126115]
 [-0.1392216 ]
 [-0.14390743]
 [-0.19852394]
 [-0.311572  ]]
Original Biases of dense_layer:
 [-0.37227145]
15018/15018 ━━━━━━━━━━━━━━━━━━━━ 34s 2ms/step - loss: 34.6749 - mae: 4.6756 - val_loss: 74.7243 - val_mae: 7.2140
Original Weights of dense_layer:
 [[-0.07476436]
 [-0.0946381 ]
 [-0.14686128]
 [-0.1417465 ]
 [-0.17892452]
 [-0.2738686 ]]
Original Biases of dense_layer:
 [-0.376102]
 9293/15018 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: 34.7122 - mae: 4.6809

KeyboardInterrupt: 

In [44]:
print(len(np.load("boards.npy")))
# 2_402_853

2402853


In [48]:
print(model.layers[0].name)

regression: lines


In [55]:
import numpy as np

dense_layer_weights = model.get_layer('output').get_weights()
weights, biases = dense_layer_weights
print("Original Weights of dense_layer:\n", weights)
print("Original Biases of dense_layer:\n", biases)


model.get_layer('output').set_weights([np.array([[1],[3],[10],[-1],[-3],[-10]]), np.array([0])])


dense_layer_weights = model.get_layer('output').get_weights()
weights, biases = dense_layer_weights
print("Original Weights of dense_layer:\n", weights)
print("Original Biases of dense_layer:\n", biases)

Original Weights of dense_layer:
 [[1.]
 [2.]
 [3.]
 [4.]
 [5.]
 [6.]]
Original Biases of dense_layer:
 [0.]
Original Weights of dense_layer:
 [[  1.]
 [  3.]
 [ 10.]
 [ -1.]
 [ -3.]
 [-10.]]
Original Biases of dense_layer:
 [0.]
